In [9]:
#!/usr/bin/env python
import cv2
import numpy as np
import os
import glob

In [10]:
# Defining the dimensions of checkerboard
CHECKERBOARD = (6,9)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

In [11]:
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
 


In [12]:
# Extracting path of individual image stored in a given directory
images = glob.glob('C:/Users/rosas/Documents/Epfl/MA_2/Design Project/cherckerboard 2/*.JPG')

for fname in images:
    try:
        img = cv2.imread(fname)
        if img is None:
            raise FileNotFoundError("Failed to load image: {}".format(fname))

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, 
                                                 cv2.CALIB_CB_ADAPTIVE_THRESH + 
                                                 cv2.CALIB_CB_FAST_CHECK + 
                                                 cv2.CALIB_CB_NORMALIZE_IMAGE)
        
        if ret == True:
            objpoints.append(objp)
            
            # Refine the pixel coordinates for given 2D points
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)
            
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

            # Create a named window with a specific size
            cv2.namedWindow('img', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('img', 800, 600)  # Set the window size to 800x600 pixels


            cv2.imshow('img', img)
            cv2.waitKey(500)  # Display the image for 0.5 seconds
    except Exception as e:
        print("Error processing image {}: {}".format(fname, e))
        continue
        
cv2.destroyAllWindows()

# Calculate the image size
h, w = img.shape[:2]


"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[3.52546709e+03 0.00000000e+00 2.80962549e+03]
 [0.00000000e+00 3.53751668e+03 2.22828710e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[-0.07186645  0.11189368  0.00865094 -0.00050088 -0.10195969]]
rvecs : 

(array([[-0.10472679],
       [-0.18744582],
       [ 1.55103168]]), array([[-0.33213939],
       [-0.48316699],
       [ 1.46859414]]), array([[-0.27580829],
       [-0.22635335],
       [ 1.5734566 ]]), array([[-0.30488867],
       [-0.07598816],
       [ 1.56177745]]), array([[-0.6478806 ],
       [ 0.03605223],
       [ 1.59263517]]), array([[0.20478504],
       [0.71822747],
       [1.40179955]]), array([[-0.0638072 ],
       [ 0.1203264 ],
       [ 1.58522092]]), array([[-0.76514674],
       [-0.74288602],
       [ 1.30762154]]), array([[-0.60899213],
       [-0.838129  ],
       [ 1.43253258]]), array([[-0.10954341],
       [-0.11448298],
       [ 1.45653388]]), array([[0.80695637],
       [0.50958524],
       [1.37762722]]), array([[0.1

In [13]:
img = cv2.imread('checkerboard/IMG_0292.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [14]:
# undistort
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

True

In [15]:
print(dst.shape)
print(dist)

(3350, 2330, 3)
[[-0.07186645  0.11189368  0.00865094 -0.00050088 -0.10195969]]


In [16]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
 
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.2159626417832498
